# Threading common Objects and Methods:

### 1. threading.Thread() Objects and Methods:
- 1.1 start()
- 1.2 run()
- 1.3 join(timeout=None)
- 1.4 is_alive()
- 1.5 getName()
- 1.6 setName(name)
- 1.7 daemon
- 1.8 name

### 1.1 Start():
- The start() method in multi-threading begins the execution of a new thread. It sets up the thread's resources, moves it from the "New" state to "Runnable," and invokes its run() method in parallel with the main thread, enabling concurrent execution.

In [5]:
import threading

# Define a thread by subclassing threading.Thread
class MyThread(threading.Thread):
    def run(self):
        print(f"Thread is running: {threading.current_thread().name}")

# Main program
thread = MyThread()
thread.start()  # Start the thread, triggering run()
print(f"\nMain thread is running: {threading.current_thread().name}")


Thread is running: Thread-4
Main thread is running: MainThread



### 1.2 run():
### What the run() Method Does?
The run() method in a thread is where the actual work or logic of the thread is defined. This can be achieved in two ways:

#### i. Target Method (Function Approach):

- You pass a function (target method) to the Thread object at creation.
- The run() method executes this target function when the thread is started.
#### ii. Custom Subclass (Class-based Approach):

- You subclass the Thread class and override its run() method.
- This provides a way to define thread-specific behavior directly within the subclass.

### How the run() Method Works?

#### i. Target Function Execution (Default run() Method):
#### Internals of Default run() Method:
- The Thread class in Python has a default run() method, implemented as follows in its source code:


In [7]:
def run(self):
    if self._target:
        self._target(*self._args, **self._kwargs)


### What this means:
- If a target function is provided during thread creation (via target argument), the run() method calls this target function i.e. the _target stores the reference of target method.
- Any arguments passed via args or kwargs are passed to the target function.

### Step-by-Step Breakdown:
a. Thread Creation:

- You create a Thread object and provide a target function (e.g., task) with optional arguments.

#### thread = threading.Thread(target=task, args=(42,))
b. Calling start():

When start() is called:
- A new thread stack is created.
- Thread stack is a memory region dedicated to a thread for managing local variables, function calls, and execution state.
- The thread transitions to the RUNNABLE state, waiting for the scheduler to pick it up.

c. Executing run():

- The start() method internally invokes the thread's run() method.

Since a target function (task) is provided, the run() method executes the following steps:
- Checks if _target is not None.
- Calls _target(*_args, **_kwargs):
Here, _target is task, _args is (42,), and _kwargs is {}.

d. Calling the Target Function:

- The task function is executed with the argument 42 in the thread's context.

In [8]:
import threading

# Define a target function
def task(arg):
    print(f"Thread logic executing with argument: {arg}")

# Create a thread with the target function
thread = threading.Thread(target=task, args=(42,))
thread.start()


Thread logic executing with argument: 42


### ii. Overridden run() Execution (Custom Subclass)
#### Internals of Overriding run()
- If you subclass Thread and override the run() method, the thread will execute the logic defined in the overridden run() instead of the default behavior.

#### What this means:
- The custom logic you define inside the overridden run() method is executed when start() is called.


In [6]:
import threading

# Define a custom thread class
class MyThread(threading.Thread):
    def run(self):
        print("Thread logic executing in overridden run() method")

# Create and start a thread
thread = MyThread()
thread.start()


Thread logic executing in overridden run() method


### Why we call start() instead of run()?

#### i. Using start() Method


In [7]:
import threading
import time

def task():
    for i in range(5):
        print(f"Task running in thread: {threading.current_thread().name}, iteration: {i}")
        time.sleep(1)

# Create a thread and start it
thread = threading.Thread(target=task)
thread.start()
thread.join()
# Main thread continues its own execution
for i in range(3):
    print(f"Main thread: {threading.current_thread().name}, iteration: {i}")
    time.sleep(1)


Task running in thread: Thread-6 (task), iteration: 0
Task running in thread: Thread-6 (task), iteration: 1
Task running in thread: Thread-6 (task), iteration: 2
Task running in thread: Thread-6 (task), iteration: 3
Task running in thread: Thread-6 (task), iteration: 4
Main thread: MainThread, iteration: 0
Main thread: MainThread, iteration: 1
Main thread: MainThread, iteration: 2


### What Happens Internally?
a. Thread Creation:

- thread = threading.Thread(target=task) creates a new Thread object.

The Thread object stores:
- task function in its _target attribute.
- Default name (Thread-1) and other thread metadata.

b. Calling start():

thread.start():
- Creates a new thread in the OS or runtime.
- Allocates a separate thread stack for the new thread.
- Transitions the thread to the RUNNABLE state.

c. Scheduler Picks the Thread:
- The thread scheduler starts executing the new thread (Thread-1) independently of the main thread.

d. Calling run() Automatically:

- The new thread invokes its run() method.
- Since target=task was provided, run() executes:
The task() function runs in the new thread's stack.

e. Parallel Execution:

- While the new thread executes task(), the main thread continues executing its loop independently.


#### ii. Using run() Method


In [8]:
import threading
import time

def task():
    for i in range(5):
        print(f"Task running in thread: {threading.current_thread().name}, iteration: {i}")
        time.sleep(1)

# Create a thread and call run() directly
thread = threading.Thread(target=task)
thread.run()  # Directly calling run()

# Main thread execution
for i in range(3):
    print(f"Main thread: {threading.current_thread().name}, iteration: {i}")
    time.sleep(1)


Task running in thread: MainThread, iteration: 0
Task running in thread: MainThread, iteration: 1
Task running in thread: MainThread, iteration: 2
Task running in thread: MainThread, iteration: 3
Task running in thread: MainThread, iteration: 4
Main thread: MainThread, iteration: 0
Main thread: MainThread, iteration: 1
Main thread: MainThread, iteration: 2


### What Happens Internally?
a. Thread Creation:

- Similar to the start() example, thread = threading.Thread(target=task) creates a thread object and stores the task function in _target.

b. Calling run():

- thread.run() does not create a new thread.
- It simply executes the task() function within the current thread (i.e., the main thread’s stack).
- The thread state remains NEW, and no thread stack is allocated for a separate execution.

c. Blocking Execution:

- The task() function runs sequentially in the main thread.
- The main thread’s execution (e.g., its loop) is blocked until task() completes.


### 1.3 join():

### What join() Does?
- Ensures Thread Completion: Blocks the calling thread until the thread on which join() is called finishes execution.
- Synchronizes Execution: Prevents premature continuation of the main thread or other threads that depend on the joined thread.

### How join() Does It?
When join() is called:
- The calling thread (e.g., the main thread) waits for the target thread (the thread on which join() is called) to finish.
- The thread scheduler monitors the state of the target thread.
- Once the target thread transitions to the TERMINATED state, the calling thread is resumed and continues execution.

In [10]:
import threading
import time

def task(name, delay):
    print(f"{name} started")
    time.sleep(delay)
    print(f"{name} finished")

# Create threads
thread1 = threading.Thread(target=task, args=("Thread-1", 2))
thread2 = threading.Thread(target=task, args=("Thread-2", 3))

# Start threads
thread1.start()
thread2.start()

# Wait for both threads to finish
thread1.join()
thread2.join()

print("All threads have finished")


Thread-1 startedThread-2 started
All threads have finished

Thread-1 finished
Thread-2 finished


## Thread Attributes
### alive: 
- Checks if the thread is still alive (i.e., still running).
### name:
- Returns the name of the thread.
### ident:
- Returns the thread's unique identifier (an integer).
### native_id:
- Returns the native OS thread ID (useful for OS-level debugging).
### daemon:
- This attribute indicates whether the thread is a daemon thread (True) or a non-daemon thread (False).

In [11]:
import threading
import time

def task():
    print(f"Child thread: {child_thread.name} - Ident: {child_thread.ident}, Native ID: {child_thread.native_id}, Daemon: {child_thread.daemon}")
    time.sleep(2)
    print(f"Child thread '{threading.current_thread().name}' completed.")

child_thread = threading.Thread(target=task, name="Child-Thread", daemon=True)

child_thread.start()

child_thread.join()

print(f"{child_thread.name} is alive after join: {child_thread.is_alive()}")


Child thread: Child-Thread - Ident: 23388, Native ID: 23388, Daemon: True
Child thread 'Child-Thread' completed.
Child-Thread is alive after join: False


### 2. threading.Timer() Objects and Methods:

- In Python's threading module, Timer objects allow you to schedule a function to be executed after a certain amount of time. It is a subclass of the Thread class and runs the specified function after the given time interval.

### Timer Objects:
Timer(interval, function, args=[], kwargs={}):
- interval: The amount of time (in seconds) to wait before executing the function.
- function: The function to be executed after the interval.
- args: A tuple of arguments to pass to the function (optional).
- kwargs: A dictionary of keyword arguments to pass to the function (optional).

### Methods:
- start(): Starts the timer; the function will be called after the specified interval.
- cancel(): Stops the timer before it has a chance to run.

In [12]:
import threading

def task():
    print("Task is executed after the timer interval.")

# Create a Timer object
timer = threading.Timer(3, task)  # 3 seconds delay before calling task()

# Start the timer
timer.start()

# Main thread waits for the timer to complete
timer.join()

print("Main thread has finished execution.")


Task is executed after the timer interval.
Main thread has finished execution.


### How it Works:
- The Timer object timer will wait for 3 seconds before calling the task function.
- After the 3-second delay, the task function is executed.
- join() is called to ensure the main thread waits for the timer to complete before it exits.

### Key Points:
- Timer allows scheduling functions to run after a delay.
- It behaves like a thread, but the primary difference is that it only executes the given function once after the delay.
- You can cancel the timer before it executes using cancel() if needed.

In [21]:
import threading

def task():
    print("This task will not be executed.")

# Create a Timer object
timer = threading.Timer(5, task)

# Start the timer
timer.start()

# Cancel the timer before it executes
timer.cancel()

print("Timer was canceled before execution.")


Timer was canceled before execution.


### Why we need start():
- Trigger Timer Execution: The start() method is necessary to begin the countdown for the scheduled task. Without calling start(), the timer won't start, and the function will not be executed after the specified interval.
- Asynchronous Execution: It starts the timer in a new thread, allowing the main program to continue running while waiting for the timer to execute the function after the given interval.

### Why we need cancel():
- Interrupt Timer Execution: The cancel() method is used to stop the timer before it executes the scheduled function. If the timer hasn't yet expired (i.e., the function hasn't been executed), calling cancel() will prevent the function from running.
- Flexibility in Scheduling: It allows flexibility, so you can dynamically cancel scheduled tasks. This is useful when the conditions for executing the function change, or you decide that the function should no longer be executed.